In [5]:
from pathlib import Path
from tqdm import tqdm

from common.text.unidecoder import _homoglyphs, _replacements

def check_text(s, homoglyphs=False) -> bool:
    """Transliterate unicode

    Args:
        s (str): unicode string
        homoglyphs (bool): prioritize translating to homoglyphs
    """
    for u in s:
        if ord(u) < 127:
            a = u
        elif homoglyphs:
            a = _homoglyphs.get(u, _replacements.get(u, None))
        else:
            a = _replacements.get(u, _homoglyphs.get(u, None))

        if a is None:
            return False
    return True


folder = Path('filelists')

speakers = {
    'lj': "0",
    'stephan': "1"
}

for file_path in tqdm(folder.iterdir(), desc='Files'):
    if file_path.suffix != '.txt':
        continue
    new_data = []
    with open(file_path, 'r') as file:
        data = file.readlines()
        for line in data:
            line = line.strip()
            if len(line) == 0:
                continue
            parts = line.split('|')
            text = parts[-2]
            if not check_text(text):
                print(f'Problem text: {text}')
                continue
            speaker_str = parts[-1]
            if speaker_str in speakers:
                parts[-1] = speakers[speaker_str]
            new_data.append('|'.join(parts) + '\n')
    with open(file_path, 'w') as file:
        file.writelines(new_data)

Files: 14it [00:00, 99.21it/s]


In [3]:
from pathlib import Path
import shutil

folder = Path('filelists')

shutil.copy('/home/server2/datasets/Stephan/meta.txt', folder / 'stephan_audio_text.txt')

PosixPath('filelists/stephan_audio_text.txt')

In [3]:
import numpy as np

with open('filelists/stephan_audio_text.txt', 'r') as file:
    data = file.readlines()
    data = [line for line in data if len(line.strip()) > 0]
np.random.shuffle(data)

val_data = data[:50]
data = data[50:]

test_data = data[:250]
data = data[250:]

train_data = data

with open('filelists/stephan_audio_text_val.txt', 'w') as file:
    file.writelines(val_data)
with open('filelists/stephan_audio_text_test.txt', 'w') as file:
    file.writelines(test_data)
with open('filelists/stephan_audio_text_train.txt', 'w') as file:
    file.writelines(train_data)

In [4]:
from pathlib import Path

l_files = [
    ('filelists/stephan_audio_text_test.txt', 'filelists/stephan_audio_pitch_text_test.txt'),
    ('filelists/stephan_audio_text_val.txt', 'filelists/stephan_audio_pitch_text_val.txt'),
    ('filelists/stephan_audio_text_train.txt', 'filelists/stephan_audio_pitch_text_train.txt')
]

for in_file, out_file in l_files:
    with open(in_file, 'r') as file:
        data = file.readlines()
        data = [line for line in data if len(line.strip()) > 0]
        new_data = []
        for line in data:
            parts = line.strip().split('|')
            wav_path = Path(parts[0])
            pitch_path = Path('pitch') / f'{wav_path.stem}.pt'
            parts.insert(1, str(pitch_path))
            new_data.append('|'.join(parts) + '\n')
    with open(out_file, 'w') as file:
        file.writelines(new_data)


In [8]:
merge_data = {
    'filelists/train.txt': [
        {
            'root': 'LJSpeech-1.1',
            'list_file': 'filelists/ljs_audio_pitch_text_train_v3.txt'
        },
        {
            'root': 'Stephan',
            'list_file': 'filelists/stephan_audio_pitch_text_train.txt'
        }
    ],
    'filelists/val.txt': [
        {
            'root': 'LJSpeech-1.1',
            'list_file': 'filelists/ljs_audio_pitch_text_val.txt'
        },
        {
            'root': 'Stephan',
            'list_file': 'filelists/stephan_audio_pitch_text_val.txt'
        }
    ],
    'filelists/test.txt': [
        {
            'root': 'LJSpeech-1.1',
            'list_file': 'filelists/ljs_audio_pitch_text_test.txt'
        },
        {
            'root': 'Stephan',
            'list_file': 'filelists/stephan_audio_pitch_text_test.txt'
        }
    ],
}

for file_name, m_data in merge_data.items():
    new_data = []
    for sub_m_data in m_data:
        with open(sub_m_data['list_file'], 'r') as file:
            data = file.readlines()
        root_folder = Path(sub_m_data['root'])
        for line in data:
            parts = line.strip().split('|')
            wav_path = Path(parts[0])
            pitch_path = Path(parts[1])
            parts[0] = str(root_folder / wav_path)
            parts[1] = str(root_folder / pitch_path)
            new_data.append('|'.join(parts) + '\n')
    with open(file_name, 'w') as file:
        file.writelines(new_data)
        